In [1]:
import sys
sys.chdir('../')

import numpy as np
import pandas as pd
import random

from sklearn.ensemble import RandomForestClassifier
from src.utils import *
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from keras.layers import Embedding, LSTM
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import losses
from keras import regularizers
from keras.constraints import max_norm
from keras.wrappers.scikit_learn import KerasClassifier

import time

from sklearn.model_selection import StratifiedKFold

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Loading data

In [2]:
Data_X_train = pd.read_csv('data/challenge_fichier_dentrees_dentrainement_challenge_nba/train.csv')
Data_Y_train = pd.read_csv('data/challenge_fichier_de_sortie_dentrainement_challenge_nba.csv', sep=';')

In [3]:
Data_X_train = feature_engineering(Data_X_train)

100%|██████████| 1440/1440 [06:09<00:00,  3.89it/s] 


In [4]:
X = Data_X_train.as_matrix()[:,1:]
Y = Data_Y_train.as_matrix()[:,1:]

# K fold functions

In [5]:
def train_and_evaluate(model, X_train, Y_train, X_val, Y_val, epochs, batch_size, verbose):
    training = model.fit(X_train, Y_train, epochs = epochs, batch_size = batch_size, verbose = verbose)
    evalu = model.evaluate(X_val, Y_val)
    
    return training.history['acc'][-1], evalu[1]

In [6]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 7)

In [7]:
def kfold_print(skf_splitter, X, Y, create_model_func, epochs, batch_size, verbose):
    accs_train = []
    accs_val = []
    for train, val in skf.split(X, Y):
        model = create_model_func()
        Y_train = np_utils.to_categorical(Y[train], 2)
        Y_val = np_utils.to_categorical(Y[val], 2)
        acc_train, acc_val = train_and_evaluate(model, 
                                                X[train], Y_train, X[val], Y_val, 
                                                epochs = epochs, batch_size = batch_size, verbose = verbose)

        accs_train.append(acc_train)
        accs_val.append(acc_val)
        print('(Training, Validation) accuracies: ({0:.2f},{1:.2f})'.format(100*acc_train, 100*acc_val))

    print('Mean Training Accuracy: {0:.2f} +/- {1:.2f}'.format(100*np.mean(accs_train), 100*np.std(accs_train)))
    print('Mean Validation Accuracy: {0:.2f} +/- {1:.2f}'.format(100*np.mean(accs_val), 100*np.std(accs_val)))

# 15sec subsequences - Aggregation = Mean

In [8]:
X_train = X.reshape((len(X), 14, 15, -1), order = 'F')
X_train = X_train.mean(axis = 2)

In [9]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(30, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 0s 156us/step
(Training, Validation) accuracies: (69.89,73.13)
2516/2516 [==============================] - 0s 142us/step
(Training, Validation) accuracies: (69.62,71.66)
2515/2515 [==============================] - 0s 164us/step
(Training, Validation) accuracies: (71.31,71.13)
2515/2515 [==============================] - 0s 167us/step
(Training, Validation) accuracies: (71.37,71.69)
2514/2514 [==============================] - 0s 184us/step
(Training, Validation) accuracies: (71.94,68.97)
Mean Training Accuracy: 70.83 +/- 0.91
Mean Validation Accuracy: 71.32 +/- 1.35


In [10]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 0s 190us/step
(Training, Validation) accuracies: (70.67,72.93)
2516/2516 [==============================] - 1s 224us/step
(Training, Validation) accuracies: (71.13,72.38)
2515/2515 [==============================] - 1s 229us/step
(Training, Validation) accuracies: (71.94,71.21)
2515/2515 [==============================] - 1s 225us/step
(Training, Validation) accuracies: (71.35,72.64)
2514/2514 [==============================] - 1s 241us/step
(Training, Validation) accuracies: (72.56,68.85)
Mean Training Accuracy: 71.53 +/- 0.66
Mean Validation Accuracy: 71.60 +/- 1.49


In [11]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(75, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 1s 254us/step
(Training, Validation) accuracies: (72.01,72.18)
2516/2516 [==============================] - 1s 264us/step
(Training, Validation) accuracies: (71.22,71.42)
2515/2515 [==============================] - 1s 288us/step
(Training, Validation) accuracies: (72.45,70.97)
2515/2515 [==============================] - 1s 289us/step
(Training, Validation) accuracies: (72.16,71.93)
2514/2514 [==============================] - 1s 303us/step
(Training, Validation) accuracies: (73.15,69.81)
Mean Training Accuracy: 72.20 +/- 0.62
Mean Validation Accuracy: 71.26 +/- 0.84


In [12]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 1s 314us/step
(Training, Validation) accuracies: (72.26,71.90)
2516/2516 [==============================] - 1s 328us/step
(Training, Validation) accuracies: (72.73,72.14)
2515/2515 [==============================] - 1s 383us/step
(Training, Validation) accuracies: (72.95,70.34)
2515/2515 [==============================] - 1s 387us/step
(Training, Validation) accuracies: (72.81,71.49)
2514/2514 [==============================] - 1s 385us/step
(Training, Validation) accuracies: (73.70,69.53)
Mean Training Accuracy: 72.89 +/- 0.47
Mean Validation Accuracy: 71.08 +/- 0.99


In [13]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 1s 419us/step
(Training, Validation) accuracies: (73.87,71.07)
2516/2516 [==============================] - 1s 409us/step
(Training, Validation) accuracies: (73.48,71.22)
2515/2515 [==============================] - 1s 436us/step
(Training, Validation) accuracies: (73.20,71.49)
2515/2515 [==============================] - 1s 450us/step
(Training, Validation) accuracies: (74.40,71.17)
2514/2514 [==============================] - 1s 443us/step
(Training, Validation) accuracies: (75.25,68.62)
Mean Training Accuracy: 74.04 +/- 0.73
Mean Validation Accuracy: 70.71 +/- 1.06


In [14]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(200, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 1s 440us/step
(Training, Validation) accuracies: (74.82,72.22)
2516/2516 [==============================] - 1s 462us/step
(Training, Validation) accuracies: (73.86,71.34)
2515/2515 [==============================] - 1s 467us/step
(Training, Validation) accuracies: (75.94,70.74)
2515/2515 [==============================] - 1s 538us/step
(Training, Validation) accuracies: (75.04,71.69)
2514/2514 [==============================] - 1s 506us/step
(Training, Validation) accuracies: (77.55,69.09)
Mean Training Accuracy: 75.44 +/- 1.24
Mean Validation Accuracy: 71.02 +/- 1.08


In [15]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(300, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 1s 510us/step
(Training, Validation) accuracies: (75.57,70.87)
2516/2516 [==============================] - 1s 537us/step
(Training, Validation) accuracies: (77.32,70.59)
2515/2515 [==============================] - 1s 543us/step
(Training, Validation) accuracies: (78.97,69.62)
2515/2515 [==============================] - 1s 545us/step
(Training, Validation) accuracies: (77.24,70.30)
2514/2514 [==============================] - 1s 570us/step
(Training, Validation) accuracies: (77.64,68.02)
Mean Training Accuracy: 77.35 +/- 1.09
Mean Validation Accuracy: 69.88 +/- 1.02


In [16]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(30, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 1s 576us/step
(Training, Validation) accuracies: (71.68,72.97)
2516/2516 [==============================] - 2s 602us/step
(Training, Validation) accuracies: (71.40,72.58)
2515/2515 [==============================] - 2s 600us/step
(Training, Validation) accuracies: (71.86,71.01)
2515/2515 [==============================] - 2s 609us/step
(Training, Validation) accuracies: (72.00,71.77)
2514/2514 [==============================] - 2s 627us/step
(Training, Validation) accuracies: (72.07,69.29)
Mean Training Accuracy: 71.80 +/- 0.24
Mean Validation Accuracy: 71.52 +/- 1.30


In [17]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 2s 621us/step
(Training, Validation) accuracies: (72.71,72.69)
2516/2516 [==============================] - 2s 653us/step
(Training, Validation) accuracies: (72.99,71.46)
2515/2515 [==============================] - 2s 672us/step
(Training, Validation) accuracies: (72.91,70.38)
2515/2515 [==============================] - 2s 662us/step
(Training, Validation) accuracies: (73.10,72.17)
2514/2514 [==============================] - 2s 700us/step
(Training, Validation) accuracies: (73.44,69.57)
Mean Training Accuracy: 73.03 +/- 0.24
Mean Validation Accuracy: 71.25 +/- 1.15


In [18]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(75, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 2s 703us/step
(Training, Validation) accuracies: (74.43,72.58)
2516/2516 [==============================] - 2s 768us/step
(Training, Validation) accuracies: (74.33,72.10)
2515/2515 [==============================] - 2s 821us/step
(Training, Validation) accuracies: (75.24,69.94)
2515/2515 [==============================] - 2s 734us/step
(Training, Validation) accuracies: (74.15,71.25)
2514/2514 [==============================] - 2s 752us/step
(Training, Validation) accuracies: (74.25,69.37)
Mean Training Accuracy: 74.48 +/- 0.39
Mean Validation Accuracy: 71.05 +/- 1.23


In [19]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 2s 757us/step
(Training, Validation) accuracies: (76.05,72.18)
2516/2516 [==============================] - 2s 770us/step
(Training, Validation) accuracies: (77.38,70.27)
2515/2515 [==============================] - 2s 780us/step
(Training, Validation) accuracies: (76.54,71.21)
2515/2515 [==============================] - 2s 790us/step
(Training, Validation) accuracies: (76.73,70.18)
2514/2514 [==============================] - 2s 818us/step
(Training, Validation) accuracies: (77.93,68.50)
Mean Training Accuracy: 76.93 +/- 0.66
Mean Validation Accuracy: 70.47 +/- 1.22


In [20]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 2s 817us/step
(Training, Validation) accuracies: (80.89,71.03)
2516/2516 [==============================] - 2s 816us/step
(Training, Validation) accuracies: (80.43,70.19)
2515/2515 [==============================] - 2s 832us/step
(Training, Validation) accuracies: (81.49,69.34)
2515/2515 [==============================] - 2s 848us/step
(Training, Validation) accuracies: (81.77,70.38)
2514/2514 [==============================] - 2s 857us/step
(Training, Validation) accuracies: (82.38,68.26)
Mean Training Accuracy: 81.39 +/- 0.68
Mean Validation Accuracy: 69.84 +/- 0.96


In [21]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(200,
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 2s 876us/step
(Training, Validation) accuracies: (84.96,70.75)
2516/2516 [==============================] - 2s 903us/step
(Training, Validation) accuracies: (85.09,70.95)
2515/2515 [==============================] - 2s 909us/step
(Training, Validation) accuracies: (84.38,68.79)
2515/2515 [==============================] - 2s 920us/step
(Training, Validation) accuracies: (86.20,68.35)
2514/2514 [==============================] - 2s 938us/step
(Training, Validation) accuracies: (85.52,67.94)
Mean Training Accuracy: 85.23 +/- 0.61
Mean Validation Accuracy: 69.35 +/- 1.25


In [22]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(300, 
                        input_shape = (14, 96)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 2s 939us/step
(Training, Validation) accuracies: (88.60,68.92)
2516/2516 [==============================] - 2s 960us/step
(Training, Validation) accuracies: (89.71,66.53)
2515/2515 [==============================] - 2s 984us/step
(Training, Validation) accuracies: (88.77,69.15)
2515/2515 [==============================] - 2s 982us/step
(Training, Validation) accuracies: (90.04,67.91)
2514/2514 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (90.01,68.42)
Mean Training Accuracy: 89.43 +/- 0.62
Mean Validation Accuracy: 68.19 +/- 0.93


# 10sec subsequences - Aggregation = Mean

In [23]:
X_train = X.reshape((len(X), 14, 10, -1), order = 'F')
X_train = X_train.mean(axis = 2)

In [24]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(30, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (69.81,72.34)
2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (70.00,72.34)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (70.44,70.62)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (70.19,71.61)
2514/2514 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.25,68.93)
Mean Training Accuracy: 70.34 +/- 0.50
Mean Validation Accuracy: 71.17 +/- 1.28


In [25]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.17,72.85)
2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.06,72.66)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.94,71.37)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.28,71.93)
2514/2514 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (72.06,69.01)
Mean Training Accuracy: 71.50 +/- 0.41
Mean Validation Accuracy: 71.56 +/- 1.38


In [26]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(75, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.83,73.25)
2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (70.58,72.18)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.97,70.78)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.20,72.41)
2514/2514 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.92,68.85)
Mean Training Accuracy: 71.50 +/- 0.54
Mean Validation Accuracy: 71.49 +/- 1.54


In [27]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.87,72.85)
2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.64,72.18)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.62,71.09)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (71.92,71.69)
2514/2514 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (72.14,69.45)
Mean Training Accuracy: 71.84 +/- 0.19
Mean Validation Accuracy: 71.45 +/- 1.16


In [28]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (72.51,72.93)
2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (72.64,71.54)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (72.93,71.05)
2515/2515 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (72.56,72.49)
2514/2514 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (73.04,68.89)
Mean Training Accuracy: 72.74 +/- 0.21
Mean Validation Accuracy: 71.38 +/- 1.41


In [29]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(200, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 3s 1ms/step
(Training, Validation) accuracies: (72.46,71.74)
2516/2516 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (73.26,72.62)
2515/2515 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (73.06,71.81)
2515/2515 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (72.58,71.41)
2514/2514 [==============================] - 7s 3ms/step
(Training, Validation) accuracies: (73.80,69.13)
Mean Training Accuracy: 73.03 +/- 0.49
Mean Validation Accuracy: 71.34 +/- 1.17


In [30]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(300, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 20,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (73.36,72.62)
2516/2516 [==============================] - 4s 1ms/step
(Training, Validation) accuracies: (74.13,72.22)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (73.29,71.13)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (72.93,71.53)
2514/2514 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (74.92,68.70)
Mean Training Accuracy: 73.72 +/- 0.71
Mean Validation Accuracy: 71.24 +/- 1.37


In [31]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(30, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (71.17,72.89)
2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (70.99,72.54)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (71.01,70.78)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (71.15,71.65)
2514/2514 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (72.49,68.97)
Mean Training Accuracy: 71.36 +/- 0.57
Mean Validation Accuracy: 71.37 +/- 1.40


In [32]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(50, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (72.50,72.85)
2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (72.43,71.94)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (72.43,70.82)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (72.28,71.61)
2514/2514 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (73.42,69.41)
Mean Training Accuracy: 72.61 +/- 0.41
Mean Validation Accuracy: 71.33 +/- 1.16


In [33]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(75, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (72.70,73.45)
2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (73.86,72.14)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (73.72,70.58)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (73.13,71.81)
2514/2514 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (73.66,69.21)
Mean Training Accuracy: 73.42 +/- 0.43
Mean Validation Accuracy: 71.44 +/- 1.44


In [34]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (73.66,72.66)
2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (74.80,72.58)
2515/2515 [==============================] - 4s 2ms/step
(Training, Validation) accuracies: (74.53,70.54)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (74.33,72.21)
2514/2514 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (75.27,70.21)
Mean Training Accuracy: 74.52 +/- 0.53
Mean Validation Accuracy: 71.64 +/- 1.05


In [35]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (75.44,71.22)
2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (77.58,71.58)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (76.07,70.62)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (76.07,71.85)
2514/2514 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (78.24,68.77)
Mean Training Accuracy: 76.68 +/- 1.05
Mean Validation Accuracy: 70.81 +/- 1.10


In [36]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(200, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (79.22,71.42)
2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (78.83,71.07)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (79.70,70.14)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (79.61,69.38)
2514/2514 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (79.16,70.05)
Mean Training Accuracy: 79.31 +/- 0.32
Mean Validation Accuracy: 70.41 +/- 0.74


In [37]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(300, 
                        input_shape = (14, 144)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (81.72,70.95)
2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (82.50,70.63)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (82.48,69.38)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (81.59,68.19)
2514/2514 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (81.95,68.74)
Mean Training Accuracy: 82.05 +/- 0.38
Mean Validation Accuracy: 69.58 +/- 1.06


# 20 sec subsequences - Aggregation = Mean

In [38]:
X_train = X.reshape((len(X), 14, 20, -1), order = 'F')
X_train = X_train.mean(axis = 2)

In [39]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (14, 72)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 75,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (71.81,72.26)
2516/2516 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (71.79,70.11)
2515/2515 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (72.49,71.25)
2515/2515 [==============================] - 17s 7ms/step
(Training, Validation) accuracies: (72.35,71.93)
2514/2514 [==============================] - 7s 3ms/step
(Training, Validation) accuracies: (72.07,69.65)
Mean Training Accuracy: 72.10 +/- 0.28
Mean Validation Accuracy: 71.04 +/- 1.01


In [40]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, 
                        input_shape = (14, 72)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 100,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (72.36,70.35)
2516/2516 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (72.74,72.06)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (72.66,70.78)
2515/2515 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (72.87,71.53)
2514/2514 [==============================] - 5s 2ms/step
(Training, Validation) accuracies: (73.57,69.29)
Mean Training Accuracy: 72.84 +/- 0.40
Mean Validation Accuracy: 70.80 +/- 0.96


In [43]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (14, 72)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 150,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 20, 32, False)

2516/2516 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (71.67,72.34)
2516/2516 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (71.17,69.91)
2515/2515 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (71.48,70.89)
2515/2515 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (71.76,71.89)
2514/2514 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (72.07,69.57)
Mean Training Accuracy: 71.63 +/- 0.30
Mean Validation Accuracy: 70.92 +/- 1.08


In [42]:
def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(150, 
                        input_shape = (14, 72)))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 100,
                        activation = 'relu'))
    lstm_model.add(Dropout(0.75))
    lstm_model.add(Dense(units = 2, 
                    activation='softmax'))
    #Construct Loss
    lstm_model.compile(loss = losses.categorical_crossentropy,
                         optimizer = 'adam',
                         metrics = ['accuracy'])
    
    return lstm_model

kfold_print(skf, X_train, Y, create_model, 30, 32, False)

2516/2516 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (73.24,71.66)
2516/2516 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (73.32,71.07)
2515/2515 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (73.48,69.74)
2515/2515 [==============================] - 6s 2ms/step
(Training, Validation) accuracies: (73.42,71.21)
2514/2514 [==============================] - 7s 3ms/step
(Training, Validation) accuracies: (75.32,68.30)
Mean Training Accuracy: 73.76 +/- 0.79
Mean Validation Accuracy: 70.40 +/- 1.23
